In [3]:
import pandas as pd
import seaborn.apionly as sns
import numpy as np
import scipy as sp 
from  matplotlib import pyplot as plt
sns.set(font_scale=1.1, style="white",  #palette="viridis", 
        rc={'font.size': 16, 'figure.figsize': (7,4), 'axes.grid': True, 'lines.linewidth':2.0, 
            'grid.color': '.8', 'grid.linewidth': 0.5,})
# np.set_printoptions(suppress=True)

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [5]:
# !pip install GPy 
# !pip install sklearn
# import GPy as gpy
# !git clone https://github.com/bestxolodec/DTR.git
# !cd DTR; git pull 

In [6]:
import sys
sys.path.append("/Users/ipaulo/yandexDisk/DIPLOMA/CODE/py-src/")

In [7]:
from itr import * 
# import rpy2.robjects as ro
# import rpy2
# ro.r.source("/Users/ipaulo/yandexDisk/DIPLOMA/CODE/src/clean_sources.R");

itr.py is imported!


## Read data

In [16]:
X, Y = np.load("../data/sarcos/Xtr.npy"),  np.load("../data/sarcos/Ytr.npy")
rng = np.random.RandomState(777)
n_train, n_test, n_val = 500, 200, 100
shuffle_idxs = np.arange(X.shape[0])
np.random.shuffle(shuffle_idxs)
n_restarts = 1
Xtr, Xts, Xvl, _ = np.split(X[shuffle_idxs, :], np.cumsum([n_train, n_test, n_val]))
Ytr, Yts, Yvl, _ = np.split(Y[shuffle_idxs, :], np.cumsum([n_train, n_test, n_val]))

itr.py is imported!


###  Simple miltioutput GP 

In [17]:
def get_mogp_score(preds, test):
    """return mean reward â aka mean squared euclidian distance"""
    if isinstance(test, dict):
        return np.square(test["optimal.treatment"] - preds).sum(axis=1).mean()
    else:
        return np.square(Yts - preds).sum(axis=1).mean()
    
def pred_value(preds, test):
    return - get_mogp_score(preds, test) 

### Baseline regression model

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.base import clone

class BaselineModel(object):
    def __init__(self, *args, **kwargs): 
        self.m = LinearRegression(*args, **kwargs)

    def fit(self, X, Y):
        self.models = [clone(self.m).fit(X, Y[:, i]) for i in range(Y.shape[1])]
        return self
        
    def predict(self, Xts):
        return np.hstack([m.predict(Xts).reshape(-1,1) for m in self.models])
    
    def get_score(self, Xts, Yts):
        return np.square(Yts - self.predict(Xts)).sum(axis=1).mean()
    
    def get_rewards_for(self, Xts, Yts):  
        return - np.log(np.square(Yts - self.predict(Xts)).sum(axis=1).reshape(-1,1))
        return - np.square(Yts - self.predict(Xts)).sum(axis=1).reshape(-1,1)
        # TODO: normal shaped alternative 

    
    def build_dataset(self, Xtr, Ytr):
        df = pd.DataFrame(np.hstack([Xtr, self.predict(Xtr), self.get_rewards_for(Xtr, Ytr)]))
        df.columns = np.hstack([["C{}".format(i) for i in range(Xtr.shape[1])], 
                                ["A{}".format(i) for i in range(Ytr.shape[1])],
                                ["R"]]) 
        return df

    def get_dataset_in_dict(self, Xtr, Ytr):
        return {"covariates": Xtr, "treatment": self.predict(Xtr), 
                "reward": self.get_rewards_for(Xtr, Ytr), "optimal.treatment": Ytr}



In [19]:
bm = BaselineModel(n_jobs=-1).fit(X, Y)
train = bm.get_dataset_in_dict(Xtr, Ytr)
test = bm.get_dataset_in_dict(Xts, Yts)

In [ ]:
%%time
s_factors_percs = np.arange(.5,1, .01)
s_factors = sp.stats.norm.ppf(s_factors_percs) # 50 factors evenly splitted
granularity = 50
fit_params = {"mean_fn": False, "n_restarts": 1, "verbose":False, 
              "robust":True, "normalize": False}
preds, v, m = fit_and_predict(train, test, granularity, s_factors, pred_value, fit_params) 

In [18]:
import joblib

In [20]:
from joblib import  Parallel, delayed

In [ ]:
delayed()

In [ ]:
Parallel(n_jobs=4)

###  Simple miltioutput GP 

In [ ]:
%%time
k = gpy.kern.RBF(21)
# m = gpy.models.GPRegression(Xtr, Ytr, kernel=k) 
m = gpy.models.SparseGPRegression(Xtr, Ytr, kernel=k, num_inducing=2000) 
m.optimize_restarts(num_restarts=n_restarts, verbose=False, robust=True);

In [ ]:
get_mogp_score(m.predict(Xts)[0], Yts) # 2k inducing, 10k train & 10k test

In [10]:
get_mogp_score(m.predict(Xts)[0], Yts) # 1k inducing, 10k train & 10k test

20.035417479278109

In [11]:
get_mogp_score(m.predict(Xts)[0], Yts) # 7k train & 10k test

21.676514221977374

In [9]:
get_mogp_score(m.predict(Xts)[0], Yts) # 5k train & 10k test

26.705097341917799

In [39]:
get_mogp_score(m.predict(Xts)[0], Yts) # 1k train & 10k test

43.73352777260969

In [30]:
get_mogp_score(m.predict(Xts)[0], Yts) # 1k inducing on 10k train & 10k test

19.895384276908555

In [26]:
get_mogp_score(m.predict(Xts)[0], Yts) # 500 inducing on 5k train & 5k test

28.481604519136983

In [24]:
get_mogp_score(m.predict(Xts)[0], Yts)

101.99679684192976

### Coregionalized regression

In [ ]:
kernel = GPy.util.multioutput.ICM(input_dim=1,num_outputs=2,kernel=GPy.kern.Bias(input_dim=1))
m = GPy.models.GPCoregionalizedRegression(X_list=[X1,X2],Y_list=[Y1,Y2],kernel=kernel)
m['.*bias.var'].constrain_fixed(1) #B.kappa now encodes the variance.
m['.*W'].constrain_fixed(0)
m.optimize()

In [ ]:
K = GPy.kern.Matern32(1)
icm = GPy.util.multioutput.ICM(input_dim=1,num_outputs=2,kernel=K)

m = GPy.models.GPCoregionalizedRegression([X1,X2],[Y1,Y2],kernel=icm)
m['.*Mat32.var'].constrain_fixed(1.) #For this kernel, B.kappa encodes the variance now.
m.optimize()

In [111]:
from itertools import  repeat

In [ ]:
n_output = 7
icm = gpy.util.multioutput.ICM(input_dim=21,num_outputs=n_output,kernel=gpy.kern.RBF(21))
m = gpy.models.GPCoregionalizedRegression(list(repeat(Xtr, n_output)), np.split(Ytr, 7, axis=1), kernel=icm)
m.optimize()
# prediction data should include number of component
m.predict(list(repeat(Xtr, n_output))) # this will fail